<a href="https://colab.research.google.com/github/HongyuJiang/LDA-Recommendation/blob/master/LDA_Recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import absolute_import, division, print_function

import numpy as np
import os
import time
import datetime


3.6.1


In [2]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
path_to_file = '/content/drive/My Drive/LDA Recommendation/access.txt'
# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8').replace('\ufeff','')
# length of text is the number of characters in it
print ('Length of text: {} characters'.format(len(text)))

Length of text: 189954288 characters


In [0]:
lines = text.split('\n')

lines = lines[1:-1]
lines_len = len(lines)
courseDict = dict()

userOperationDict = dict()

for index in range(lines_len):
  meta = lines[index].split(',')
  user = meta[0]
  courseDetail = meta[1].replace('http://www.xuetangx.com/courses/', '')
  
  char = '/'
  
  if courseDetail.find('%') != -1:
    char = '%'
  
  linkSegment = courseDetail.split(char)
  deep = len(linkSegment)
  
  courseName = ''
  
  if len(linkSegment[0].split(':')) > 1:
    courseName = ('+'.join(linkSegment[0].split('+')[0:2])).split(':')[1]
  else:
    courseName = '+'.join(linkSegment[0].split('+')[0:2])
      
  courseDict[courseName] = 1
  
  courseChild = linkSegment[1]
  
  Optype = 'watch'
  
  if deep == 2 and courseChild == 'about':
    Optype = 'intro'
  
  if user in userOperationDict:
    if courseName in userOperationDict[user]:
      if Optype in userOperationDict[user][courseName]:
        userOperationDict[user][courseName][Optype] += 1
      else:
        userOperationDict[user][courseName][Optype] = 1
    else:
      userOperationDict[user][courseName] = dict()
      userOperationDict[user][courseName][Optype] = 1
  else:
    userOperationDict[user] = dict()
    userOperationDict[user][courseName] = dict()
    userOperationDict[user][courseName][Optype] = 1
        


In [0]:
courseDictList = list(courseDict.keys())

output = open('/content/drive/My Drive/LDA Recommendation/courses1.csv','w')

for index in range(len(courseDictList)):
  output.write(courseDictList[index] + '\n')

In [0]:
import json

courseInfoDict = dict()

path_to_file = '/content/drive/My Drive/LDA Recommendation/course.txt'

with open(path_to_file) as f:
    for line in f:
      meta = json.loads(line)
      #print(json.dumps(meta, sort_keys=True, indent=4))
      courseInfoDict[meta['course_id'].replace('/','+')] = 1
      break
#print(len(courseInfoDict))
      

In [0]:
path_to_file = '/content/drive/My Drive/LDA Recommendation/captionmap.json'

captionDict = dict()

with open(path_to_file) as f:
  data = json.loads(f.read())
  for key in data:
    #print(key)
    courseName = '+'.join(key.split('/')[2:4])
    caption = data[key]
    captionDict[courseName] = dict()
    captionDict[courseName]['caption'] = caption

In [8]:
import jieba
import jieba.posseg
import jieba.analyse
import re

jieba.analyse.set_stop_words('/content/drive/My Drive/LDA Recommendation/stopword.txt') 

def cut_words_with_pos(text):
    seg = jieba.posseg.cut(text)
    res = []
    for i in seg:
        if i.flag in ["a", "v", "x", "n", "an", "vn", "nz", "nt", "nr"] and is_fine_word(i.word):
            res.append(i.word)
    return list(res)
  
def is_fine_word(word, min_length=2):
    rule = re.compile(r"^[\u4e00-\u9fa5]+$")
    if len(word) >= min_length and re.search(rule, word):
        return True
    else:
        return False
      
sentences = []

for key in captionDict:
  caption = captionDict[key]['caption']
  seg_list = cut_words_with_pos(caption.strip())
  sentences.append(' '.join(seg_list))

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.842 seconds.
Prefix dict has been built succesfully.


In [9]:
courseList = list(captionDict.keys())

print(courseList)

output = open('/content/drive/My Drive/LDA Recommendation/courses2.csv','w')

for index in range(len(courseList)):
  output.write(courseList[index] + '\n')

['TsinghuaX+80515522X', 'PEC+20220214X', 'TsinghuaX+10421102X', 'TSINGHUA+10610193X', 'TSINGHUA+20220214X', 'MITx+6_00_2x', 'GZUDS+DSA2015001', 'XJTUSPOC+CHEM2009', 'SDSZ+30700313x', 'TSINGHUA+10610224X', 'TsinghuaX+30240184_2015WV', 'SUDA+CAR101', 'PEC+20220214X_6', 'NJU+ELEC002', 'TSINGHUA+10430484_2015X', 'XJTUSPOC+COMP1023', 'USTC+10800032X', 'TSINGHUA+30640014X', 'FZU+CAH01', 'TSINGHUA+70120073_2015X', 'TsinghuaX+10800032_2015X', 'TsinghuaX+20240103X', 'USTC+00612642X', 'TsinghuaX+20250103X', 'TsinghuaX+00720091_2015X', 'JXUFE+30640014X', 'TsinghuaX+70240183x', 'NJU+ELEC003', 'TsinghuaX+30240184X', 'SEU+00690803', 'TsinghuaX+00690242_2X', 'TsinghuaX+60240013_2015X', 'WellesleyX+ANTH207x', 'MITx+15_390x', 'USTC+20220214X', 'TsinghuaX+AP000002X', 'TsinghuaX+80150193X', 'NEU+80000902X', 'HVMOOC+sict_01', 'JLU+DKW004', 'TsinghuaX+80000901_2X', 'shanximooc+ENG001', 'MITx+6_041x', 'shanximooc+CHEM2009', 'TsinghuaX+10421084X', 'TsinghuaX+00612642X', 'BIT+PHY1701702', 'JLU+DKW001', 'PEC+2

In [0]:
#!pip uninstall pbr
#!pip uninstall lda
#!pip install lda


In [10]:
import matplotlib
import scipy
import matplotlib.pyplot as plt
from sklearn import feature_extraction  
from sklearn.feature_extraction.text import TfidfTransformer  
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
import seaborn as sns
import lda

corpus = sentences
  
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(corpus)
weight = X.toarray()

def plot_topic_words(topic_word):
    plt.figure(figsize=(8, 9))
    # f, ax = plt.subplots(5, 1, sharex=True)
    for i, k in enumerate([0, 5, 9, 14, 19]):
        ax = plt.subplot(5, 1, i+1)
        ax.plot(topic_word[k, :], 'r-')
        ax.set_xlim(-50, 4350)   # [0,4258]
        ax.set_ylim(0, 0.08)
        ax.set_ylabel(u"概率")
        ax.set_title(u"主题 {}".format(k))
    plt.xlabel(u"词", fontsize=14)
    plt.tight_layout()
    plt.suptitle(u'主题的词分布', fontsize=18)
    plt.subplots_adjust(top=0.9)
    plt.show()

def plot_topic(doc_topic):
    f, ax = plt.subplots(figsize=(30, 4))
    cmap = sns.cubehelix_palette(start=1, rot=3, gamma=0.8, as_cmap=True)
    sns.heatmap(doc_topic.T, linewidths=0.05, ax=ax)
    ax.set_title('Chat Topic in Each Course')
    ax.set_xlabel('Stage')
    ax.set_ylabel('Topic')
    plt.show()
    #f.savefig('output/topic_heatmap.jpg', bbox_inches='tight')


def lda_train(weight, vectorizer):
    model = lda.LDA(n_topics=50, n_iter=100, random_state=1)
    model.fit(weight)

    doc_num = len(weight)
    topic_word = model.topic_word_
    vocab = vectorizer.get_feature_names()
    titles = ["Stage{}".format(i) for i in range(1, doc_num + 1)]

    n_top_words = 20
    #for i, topic_dist in enumerate(topic_word):
      #print(np.sort(topic_dist))
        #topic_words = np.array(vocab)[np.argsort(topic_dist)][:-(n_top_words + 1):-1]
        #print('Topic {}: {}'.format(i, ' '.join(topic_words)))

    doc_topic = model.doc_topic_
    
    for index in range(len(doc_topic)):
      score = doc_topic[index]
      course = courseList[index]
      captionDict[course]['score'] = score
   
    #print(doc_topic, type(doc_topic))
    #plot_topic(doc_topic)
    #for i in range(doc_num):
    #    print("{} (top topic: {})".format(titles[i], np.argsort(doc_topic[i])[:-4:-1]))

lda_train(weight, vectorizer)



INFO:lda:n_documents: 331
INFO:lda:vocab_size: 12710
INFO:lda:n_words: 99244
INFO:lda:n_topics: 50
INFO:lda:n_iter: 100
INFO:lda:<0> log likelihood: -1370782
INFO:lda:<10> log likelihood: -835754
INFO:lda:<20> log likelihood: -804056
INFO:lda:<30> log likelihood: -789096
INFO:lda:<40> log likelihood: -780097
INFO:lda:<50> log likelihood: -774563
INFO:lda:<60> log likelihood: -770343
INFO:lda:<70> log likelihood: -766041
INFO:lda:<80> log likelihood: -763492
INFO:lda:<90> log likelihood: -761279
INFO:lda:<99> log likelihood: -758456


In [12]:
from sklearn.preprocessing import normalize

watchWeight = 1
infoWeight = 0.3

userTopicVectorDict = dict()

def vecAdd(vec1, vec2):
  
  vec3 = vec1
  len1 = len(vec1)
  len2 = len(vec2)
  if len1 == len2:
    for i in range(len1):
      vec3[i] += vec2[i]
  else:
    print(len1, len2)
  return vec3

def vecMutify(vec1, scale):
  
  vec2 = vec1
  len1 = len(vec1)
  
  for i in range(len1):
    vec2[i] *= scale

  return vec2

def normalized(a, axis=-1, order=2):
    l2 = np.atleast_1d(np.linalg.norm(a, order, axis))
    l2[l2==0] = 1
    return a / np.expand_dims(l2, axis)

for user in userOperationDict:
  
  userTopicVector = []
  
  for i in range(50):
    userTopicVector.append(0)
    
  userTopicVectorDict[user] = userTopicVector

  for course in userOperationDict[user]:
    
    watchNum = 0
    if 'watch' in userOperationDict[user][course]:
      watchNum = userOperationDict[user][course]['watch']
      
    introNum = 0
    if 'intro' in userOperationDict[user][course]:
      introNum = userOperationDict[user][course]['intro']
    
    #print(course)
    if course in captionDict:
      
      scale = watchNum * watchWeight + introNum * infoWeight
      
      courseTopicVector = vecMutify(captionDict[course]['score'], scale)
      #print(userTopicVectorDict[user])
      userTopicVectorDict[user] = vecAdd(userTopicVectorDict[user], courseTopicVector)
    
for user in userTopicVectorDict:
  
  vecSum = 0
  
  for index in range(50):
    vecSum += userTopicVectorDict[user][index]
  
  if vecSum > 0:
    x = userTopicVectorDict[user]
    userTopicVectorDict[user] = normalized(x)
  

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: RuntimeWarning:

overflow encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:33: RuntimeWarning:

invalid value encountered in true_divide

/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:2286: RuntimeWarning:

overflow encountered in multiply

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:68: RuntimeWarning:

overflow encountered in double_scalars

/usr/local/lib/python3.6/dist-packages/numpy/linalg/linalg.py:2287: RuntimeWarning:

overflow encountered in reduce



In [0]:
from scipy import spatial

distanceDict = dict()

userNum = len(userTopicVectorDict)

counter = 0

for userA in userTopicVectorDict:
  
  counter += 1
  
  if counter % 100 == 0:
    print(counter / userNum)
    
  for userB in userTopicVectorDict:
    if userA != userB:
      
      head = userA + '|' + userB
      vecA = userTopicVectorDict[userA]
      vecB = userTopicVectorDict[userB]
      
      sim = 1 - spatial.distance.cosine(vecA, vecB)
      
      distanceDict[head] = sim
      
print(distanceDict)

/usr/local/lib/python3.6/dist-packages/scipy/spatial/distance.py:698: RuntimeWarning:

invalid value encountered in double_scalars



In [0]:
output = open('/content/drive/My Drive/LDA Recommendation/simResult.csv','w')

for pair in distanceDict:
  val = distanceDict[pair]
  userA = pair.split('|')[0]
  userB = pair.split('|')[1]
  output.write(userA + ',' + userB + ',' + str(val) + '\n')